In [1]:
import pandas as pd
import seaborn as sns
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from sklearn.model_selection import train_test_split
import numpy as np
from datetime import date

In [2]:
# Load Train Data
train = pd.read_csv('../data/raw/train_aggr.csv',sep=';')

In [3]:
# Load Submit Data
submittion = pd.read_csv('../data/raw/ejemploRespuesta.csv')

In [4]:
submittion.columns = ['id_pos','unidades_pred']

In [6]:
train['fecha_venta_norm'] = pd.to_datetime(train['fecha_venta_norm'])

In [7]:
train['fecha_venta_norm'] = train['fecha_venta_norm'].dt.date

In [8]:
train['year'] = pd.to_datetime(train.fecha_venta_norm).dt.year
train['month'] = pd.to_datetime(train.fecha_venta_norm).dt.month

In [9]:
# Filtramos los meses que consideramos buenos para el entrenamiento (11 y 12)
train = train[train.fecha_venta_norm.isin([date(2012, 11, 1),date(2012, 12, 1),
                                                date(2013, 11, 1), date(2013, 12, 1), date(2014, 11, 1)])]

In [11]:
# Filtramos los meses que consideramos buenos para el entrenamiento (11 y 12)
test = train[train.fecha_venta_norm.isin([date(2014, 11, 1)])]

In [12]:
train.shape

(33960, 38)

In [13]:
test.shape

(5286, 38)

In [ ]:
train.sort_values(['id_pos','fecha_venta_norm'], ascending=True, inplace=True)

In [ ]:
train['unidades_target'] = train.groupby(['id_pos'])['unidades'].shift(-1)

In [18]:
rawdata.head(10)

,id_pos,fecha_venta_norm,canal,unidades,competidores,ingreso_mediana,ingreso_promedio,densidad_poblacional,pct_0a5,pct_5a9,...,pct_secundario,pct_master,pct_bicicleta,pct_omnibus,pct_subtes,pct_taxi,pct_caminata,mediana_valor_hogar,year,month
0,3142,2012-12-01,ALMACEN,12,12.0,50395.31667,68263.48667,17239.673330,6.796667,6.110000,...,27.63,7.396667,0.260000,6.733333,0.72,0.210,3.793333,252050.3467,2012,12
1,3142,2013-12-01,ALMACEN,7,12.0,50395.31667,68263.48667,17239.673330,6.796667,6.110000,...,27.63,7.396667,0.260000,6.733333,0.72,0.210,3.793333,252050.3467,2013,12
2,3143,2013-11-01,ALMACEN,2,33.0,55256.52333,70989.74667,67764.126670,6.316667,5.903333,...,29.50,7.560000,0.366667,8.410000,22.81,0.240,3.006667,391646.2067,2013,11
3,3143,2013-12-01,ALMACEN,47,33.0,55256.52333,70989.74667,67764.126670,6.316667,5.903333,...,29.50,7.560000,0.366667,8.410000,22.81,0.240,3.006667,391646.2067,2013,12
4,3144,2012-11-01,ALMACEN,6,38.0,43437.32000,58236.36500,8799.223333,7.270000,7.240000,...,28.13,6.546667,0.183333,1.466667,0.00,0.025,1.518333,137570.5233,2012,11
5,3144,2012-12-01,ALMACEN,41,38.0,43437.32000,58236.36500,8799.223333,7.270000,7.240000,...,28.13,6.546667,0.183333,1.466667,0.00,0.025,1.518333,137570.5233,2012,12
6,3144,2013-11-01,ALMACEN,4,38.0,43437.32000,58236.36500,8799.223333,7.270000,7.240000,...,28.13,6.546667,0.183333,1.466667,0.00,0.025,1.518333,137570.5233,2013,11
7,3144,2013-12-01,ALMACEN,71,38.0,43437.32000,58236.36500,8799.223333,7.270000,7.240000,...,28.13,6.546667,0.183333,1.466667,0.00,0.025,1.518333,137570.5233,2013,12
8,3146,2012-12-01,ALMACEN,7,5.0,24541.12000,38500.06000,9049.910000,5.310000,4.650000,...,31.23,3.880000,0.420000,0.410000,0.00,0.000,7.590000,112830.8300,2012,12
9,3146,2013-12-01,ALMACEN,7,5.0,24541.12000,38500.06000,9049.910000,5.310000,4.650000,...,31.23,3.880000,0.420000,0.410000,0.00,0.000,7.590000,112830.8300,2013,12


In [19]:
train = train[(rawdata.month == 11) & (train.unidades_one_periode_before.isnull()!=True)]

AttributeError: 'DataFrame' object has no attribute 'unidades_one_periode_before'

In [43]:
predictors = ['id_pos','unidades', 'unidades_target','canal', 'competidores',
       'ingreso_mediana', 'densidad_poblacional',
       'pct_0a5', 'pct_5a9', 'pct_10a14', 'pct_15a19', 'pct_20a24',
       'pct_25a29', 'pct_30a34', 'pct_35a39', 'pct_40a44', 'pct_45a49',
       'pct_50a54', 'pct_55a59', 'pct_60a64', 'pct_65a69', 'pct_70a74',
       'pct_75a79', 'pct_80a84', 'pct_85ainf', 'pct_bachelors',
       'pct_doctorados', 'pct_secundario', 'pct_master', 'pct_bicicleta',
       'pct_omnibus', 'pct_subtes', 'pct_taxi', 'pct_caminata',
       'mediana_valor_hogar']

In [44]:
X = rawdata[predictors].copy()

In [45]:
y = rawdata.unidades_target

In [46]:
X.drop('unidades_target', inplace=True, axis=1)

In [47]:
# fill Nans
for c in X.columns[X.dtypes != 'object']:
    X[c].fillna(X[c].median(), inplace=True)

In [48]:
# Label encoder
for c in X.columns[X.dtypes == 'object']:
    X[c] = X[c].factorize()[0]

In [49]:
X = sm.add_constant(X)

/opt/anaconda3/envs/signalenv/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


In [50]:
model = sm.OLS(y, X).fit()
predictions = model.predict(X) # make the predictions by the model

# Print out the statistics
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:        unidades_target   R-squared:                       0.287
Model:                            OLS   Adj. R-squared:                  0.285
Method:                 Least Squares   F-statistic:                     118.4
Date:                Mon, 17 Jun 2019   Prob (F-statistic):               0.00
Time:                        16:31:45   Log-Likelihood:                -45878.
No. Observations:               10034   AIC:                         9.183e+04
Df Residuals:                    9999   BIC:                         9.208e+04
Df Model:                          34                                         
Covariance Type:            nonrobust                                         
========================================================================================
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
const                   52.6055    158.551      0.332      0.740    -258.186     363.397
id_pos                  -0.0018   4.82e-05    -37.125      0.000      -0.002      -0.002
unidades                 1.3540      0.034     39.509      0.000       1.287       1.421
canal                  -16.1352      2.904     -5.557      0.000     -21.827     -10.443
competidores            -0.0043      0.006     -0.747      0.455      -0.016       0.007
ingreso_mediana       -5.33e-06   3.98e-05     -0.134      0.893   -8.33e-05    7.27e-05
densidad_poblacional  1.161e-05   3.76e-05      0.309      0.757   -6.21e-05    8.53e-05
pct_0a5                  0.0607      1.611      0.038      0.970      -3.098       3.219
pct_5a9                 -1.2601      1.769     -0.712      0.476      -4.728       2.208
pct_10a14                0.4725      1.622      0.291      0.771      -2.706       3.651
pct_15a19                0.4199      1.597      0.263      0.793      -2.710       3.550
pct_20a24               -0.3191      1.593     -0.200      0.841      -3.442       2.804
pct_25a29               -0.3237      1.610     -0.201      0.841      -3.480       2.833
pct_30a34                0.2510      1.667      0.151      0.880      -3.016       3.518
pct_35a39               -0.2725      1.689     -0.161      0.872      -3.583       3.038
pct_40a44                0.0528      1.683      0.031      0.975      -3.246       3.351
pct_45a49               -0.6935      1.693     -0.410      0.682      -4.011       2.625
pct_50a54               -0.1352      1.673     -0.081      0.936      -3.415       3.145
pct_55a59               -0.0857      1.669     -0.051      0.959      -3.358       3.186
pct_60a64               -0.2554      1.675     -0.153      0.879      -3.538       3.027
pct_65a69               -0.1347      1.677     -0.080      0.936      -3.422       3.153
pct_70a74               -0.3076      1.722     -0.179      0.858      -3.683       3.068
pct_75a79                1.2222      1.799      0.679      0.497      -2.305       4.749
pct_80a84               -1.0516      1.835     -0.573      0.567      -4.649       2.546
pct_85ainf              -0.0815      1.641     -0.050      0.960      -3.297       3.134
pct_bachelors            0.0863      0.100      0.860      0.390      -0.110       0.283
pct_doctorados          -0.2212      0.400     -0.553      0.581      -1.006       0.564
pct_secundario           0.0649      0.074      0.875      0.381      -0.080       0.210
pct_master              -0.0066      0.187     -0.035      0.972      -0.373       0.360
pct_bicicleta            0.0675      0.333      0.203      0.839      -0.585       0.720
pct_omnibus              0.1013      0.110      0.918      0.359      -0.115       0.318
pct_subtes              -0.0552      0.092     -0.598    

In [51]:
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:        unidades_target   R-squared:                       0.287
Model:                            OLS   Adj. R-squared:                  0.285
Method:                 Least Squares   F-statistic:                     118.4
Date:                Mon, 17 Jun 2019   Prob (F-statistic):               0.00
Time:                        16:32:28   Log-Likelihood:                -45878.
No. Observations:               10034   AIC:                         9.183e+04
Df Residuals:                    9999   BIC:                         9.208e+04
Df Model:                          34                                         
Covariance Type:            nonrobust                                         
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
const                   52.6055 

#### Linear Regression

In [52]:
X.drop('const', inplace=True, axis=1)

In [53]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [54]:
model = LinearRegression(n_jobs=4).fit(X_train, y_train)


In [55]:
r_sq = model.score(X_train, y_train)
print('coefficient of determination:', r_sq)

print('intercept:', model.intercept_)

print('slope:', model.coef_)

coefficient of determination: 0.28525809495127674
intercept: 23.728950820806787
slope: [-1.74227259e-03  1.36756506e+00 -1.51293411e+01 -2.03043439e-04
 -5.81233475e-06  1.42406550e-05 -1.59919947e-01 -1.40724818e-01
  1.51913722e-01  7.59579692e-01 -4.39882439e-02 -1.11824125e-02
  5.23245333e-01 -7.27602530e-02  5.44906415e-01 -4.00292975e-01
  5.88089763e-01  2.55096935e-02  1.27234295e-01 -6.67671232e-01
  2.17607423e-01  1.99444993e+00 -1.39537765e+00  3.64688199e-01
  3.06078889e-02  1.35505109e-01  8.53443232e-02 -3.62644963e-02
 -4.78441235e-03 -4.12809891e-02 -1.32451867e-01  9.76468993e-01
 -1.26717440e-01  9.77888122e-06]


In [56]:
y_pred = model.predict(X_test)

In [57]:
from sklearn.metrics import mean_absolute_error

In [58]:
from sklearn.metrics import mean_squared_error

In [59]:
mean_absolute_error(y_test, y_pred)

17.363484992926228

In [60]:
mean_squared_error(y_test, y_pred)

527.7745462738316

In [61]:
np.median(y_pred)

35.283469415783195

### Train Final Model

In [103]:
model = LinearRegression(n_jobs=4).fit(X, y)

In [104]:
r_sq = model.score(X_train, y_train)
print('coefficient of determination:', r_sq)

print('intercept:', model.intercept_)

print('slope:', model.coef_)

coefficient of determination: 0.0007475499909412875
intercept: -1.6365175382024901
slope: [ 8.70905905e-01 -1.12537594e-04 -1.09354694e-06 -3.07725037e-07
  3.28578227e-02  9.40190156e-03  3.46916655e-02  5.30874599e-02
  1.95930534e-02  4.02258919e-02  2.93068865e-02  2.66377692e-02
  2.53146258e-02  3.17748793e-02  4.45670177e-02  1.92741333e-02
  3.72199997e-02  6.98816004e-03  3.06150826e-02  6.65113521e-02
  4.60427094e-02  8.63861108e-03  2.67869965e-03 -1.20443590e-04
  3.58852659e-03  5.39232994e-03  2.14770958e-02 -1.63395871e-04
 -6.64356302e-04  1.01362499e-02 -4.33665929e-03  3.92879130e-08]


In [105]:
y_pred = model.predict(X)

In [106]:
mean_absolute_error(y, y_pred)

1.574054994556819